In [53]:
import pickle

import pandas as pd
import numpy as np

import re
import matplotlib.pyplot as plt
import seaborn as sns


# lib nltk
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

# lib lemmatization
#import spacy
#spacy.cli.download("pt_core_news_sm")
#import pt_core_news_sm
#spc_pt = pt_core_news_sm.load()


# lib n-grams
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# # lib WordCloud
# #from wordcloud import WordCloud

# # lib modelagem
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
df= pd.read_csv('imdb-reviews-pt-br.csv')

In [4]:
df.sample(5)

,id,text_en,text_pt,sentiment
41582,41584,"This is an above average Jackie Chan flick, du...","Este é um filme de Jackie Chan acima da média,...",pos
42639,42641,"It was only on my second viewing, years later,...","Foi somente na minha segunda exibição, anos de...",pos
20736,20738,I was very impressed with this film from newco...,Fiquei muito impressionado com este filme do r...,pos
26154,26156,I felt Rancid Aluminium was a complete waste o...,Eu senti que o Rancid Aluminum era um completo...,neg
44005,44007,Its really annoying when good movies like this...,É realmente irritante quando bons filmes como ...,pos


In [48]:
del df['id']
del df['text_en']
df

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",pos
49455,O enredo teve algumas reviravoltas infelizes e...,pos
49456,Estou espantado com a forma como este filme e ...,pos
49457,A Christmas Together realmente veio antes do m...,pos


In [49]:
df.isnull().sum()

text_pt      0
sentiment    0
dtype: int64

In [50]:
duplicados = round(sum(df.duplicated("text_pt"))/len(df)*100, 2)
print(f"Reviews com id duplicado: {duplicados}%.")

Reviews com id duplicado: 0.84%.


In [10]:
df.drop_duplicates("text_pt", inplace = True) # remove os duplicados
df

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",pos
49455,O enredo teve algumas reviravoltas infelizes e...,pos
49456,Estou espantado com a forma como este filme e ...,pos
49457,A Christmas Together realmente veio antes do m...,pos


In [51]:
df2=df.sample(2500)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 3459 to 5900
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_pt    2500 non-null   object
 1   sentiment  2500 non-null   object
dtypes: object(2)
memory usage: 58.6+ KB


In [54]:
enc= LabelEncoder()
sentimentos = enc.fit_transform(df2['sentiment'])
set(sentimentos)

{0, 1}

In [55]:
df2['sentiment'] = sentimentos
df2.head()

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de 200 pessoas que tivera...,1
43489,Minha irmã mais velha nasceu em março de 1985 ...,1


In [56]:
stopwords_pt = stopwords.words("portuguese")

In [57]:
stopwords_pt

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

In [58]:
stopwords_pt.remove('não')
stopwords_pt.remove('nem')

In [59]:
def re_breakline(text_list):
    return [re.sub('[\n\r]', ' ', r) for r in text_list]
reviews = list(df2['text_pt'].values)
reviews_breakline = re_breakline(reviews)
df2['text_pt'] = reviews_breakline
df2


,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de 200 pessoas que tivera...,1
43489,Minha irmã mais velha nasceu em março de 1985 ...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [60]:
def re_hiperlinks(text_list):
    pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return [re.sub(pattern, ' link ', r) for r in text_list]
reviews_hiperlinks = re_hiperlinks(reviews_breakline)
df2['text_pt'] = reviews_hiperlinks
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de 200 pessoas que tivera...,1
43489,Minha irmã mais velha nasceu em março de 1985 ...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [61]:
def re_dates(text_list):
    pattern = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}'
    return [re.sub(pattern, ' data ', r) for r in text_list]
reviews_dates = re_dates(reviews_breakline)
df2['text_pt'] = reviews_dates
df2


,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de 200 pessoas que tivera...,1
43489,Minha irmã mais velha nasceu em março de 1985 ...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [62]:
def re_money(text_list):
    pattern = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+'
    return [re.sub(pattern, ' dinheiro ', r) for r in text_list]
reviews_money = re_money(reviews_dates)
df2['text_pt'] = reviews_money
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de 200 pessoas que tivera...,1
43489,Minha irmã mais velha nasceu em março de 1985 ...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [63]:
def re_numbers(text_list):
     return [re.sub('[0-9]+', ' numero ', r) for r in text_list]
reviews_numbers = re_numbers(reviews_money)
df2['text_pt'] = reviews_numbers
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de numero pessoas que t...,1
43489,Minha irmã mais velha nasceu em março de nume...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [64]:
def re_negation(text_list):
     return [re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', ' negação ', r) for r in text_list]
reviews_negation = re_negation(reviews_numbers)
df2['text_pt'] = reviews_negation
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz-se que este é o primeiro polonês ocidental...,0
33962,"Quer dizer, eu pensei ter ouvido esse diálogo ...",0
45459,Eu era uma das cerca de numero pessoas que t...,1
43489,Minha irmã mais velha nasceu em março de nume...,1
...,...,...
20963,"Ei, agora, eu nunca coloquei os olhos em um qu...",1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,"Embora as peças sejam desiguais, esta coleção ...",1
34410,"E eu pensei que The Beach era ruim, com a dife...",0


In [65]:
def re_special_chars(text_list):
    return [re.sub('\W', ' ', r) for r in text_list]
reviews_special_chars = re_special_chars(reviews_negation)
df2['text_pt'] = reviews_special_chars
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz se que este é o primeiro polonês ocidental...,0
33962,Quer dizer eu pensei ter ouvido esse diálogo ...,0
45459,Eu era uma das cerca de numero pessoas que t...,1
43489,Minha irmã mais velha nasceu em março de nume...,1
...,...,...
20963,Ei agora eu nunca coloquei os olhos em um qu...,1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,Embora as peças sejam desiguais esta coleção ...,1
34410,E eu pensei que The Beach era ruim com a dife...,0


In [66]:
def re_whitespaces(text_list):
    white_spaces = [re.sub('\s+', ' ', r) for r in text_list]
    white_spaces_end = [re.sub('[ \t]+$', '', r) for r in white_spaces]
    return white_spaces_end
reviews_whitespaces = re_whitespaces(reviews_special_chars)
df2['text_pt'] = reviews_whitespaces
df2

,text_pt,sentiment
3459,Cronica de un desayuno combina os piores defei...,0
6722,Diz se que este é o primeiro polonês ocidental...,0
33962,Quer dizer eu pensei ter ouvido esse diálogo n...,0
45459,Eu era uma das cerca de numero pessoas que tiv...,1
43489,Minha irmã mais velha nasceu em março de numer...,1
...,...,...
20963,Ei agora eu nunca coloquei os olhos em um quad...,1
33634,Escrevi pelo menos meia dúzia de comentários m...,0
48152,Embora as peças sejam desiguais esta coleção d...,1
34410,E eu pensei que The Beach era ruim com a difer...,0


In [67]:
def stopwords_removal(text, cached_stopwords=stopwords.words('portuguese')):
    return [c.lower() for c in text.split() if c.lower() not in cached_stopwords]
reviews_stopwords = [' '.join(stopwords_removal(review)) for review in reviews_whitespaces]
df2['text_pt'] = reviews_stopwords
df2

,text_pt,sentiment
3459,cronica un desayuno combina piores defeitos ci...,0
6722,diz primeiro polonês ocidental escrito dirigid...,0
33962,quer dizer pensei ter ouvido diálogo filme tão...,0
45459,cerca numero pessoas sorte ver filme filme sta...,1
43489,irmã velha nasceu março numero paralisia cereb...,1
...,...,...
20963,ei agora nunca coloquei olhos quadrinho mangá ...,1
33634,escrevi menos meia dúzia comentários mordazes ...,0
48152,embora peças desiguais coleção numero curtas m...,1
34410,pensei the beach ruim diferença filme maiores ...,0


In [68]:
def stemming_process(text, stemmer=RSLPStemmer()):
    return [stemmer.stem(c) for c in text.split()]
reviews_stemmer = [' '.join(stemming_process(review)) for review in reviews_stopwords]
df2['text_pt'] = reviews_stemmer

In [69]:
df2

,text_pt,sentiment
3459,cron un desayun combin pi defeit cinem mexic f...,0
6722,diz prim polon ocident escrit dirig piotr ukla...,0
33962,qu diz pens ter ouv diálog film tão ruim lug n...,0
45459,cerc numer pesso sort ver film film stardust s...,1
43489,irmã velh nasc març numer paralis cerebr numer...,1
...,...,...
20963,ei agor nunc coloq olh quadr mang aparent film...,1
33634,escrev menos mei dúz coment mordaz dest pequen...,0
48152,emb peç desig coleç numer curt metr verdade ex...,1
34410,pens the beach ruim diferenç film mai at temp ...,0


In [70]:
X=df2.text_pt
y=df2.sentiment

In [71]:
#Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test= train_test_split(X, y)

In [72]:
#treinando o modelo
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
# running the grid search takes a long time because of the
# relatively large grid and the inclusion of trigrams
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
#param_grid = {"logisticregression__C": [0.03],
  #"tfidfvectorizer__ngram_range": [(1, 1)]}
  "tfidfvectorizer__ngram_range": [(1,1),(1, 2),(1,3)]} #O trigram nao garantiu o melhor desempenho do modelo preditivo
grid = GridSearchCV(pipe, param_grid, cv=5)
grid2= grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

Best cross-validation score: 0.84
Best parameters:
{'logisticregression__C': 10, 'tfidfvectorizer__ngram_range': (1, 2)}


In [73]:
#treinando o modelo com os melhores parametros para salvar o modelo em pkl
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
# running the grid search takes a long time because of the
# relatively large grid and the inclusion of trigrams
#param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
param_grid = {"logisticregression__C": [10],
  "tfidfvectorizer__ngram_range": [(1, 2)]}
  #"tfidfvectorizer__ngram_range": [(1,1),(1, 2),(1,3)]} #O trigram nao garantiu o melhor desempenho do modelo preditivo
grid = GridSearchCV(pipe, param_grid, cv=5)
grid2= grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

Best cross-validation score: 0.84
Best parameters:
{'logisticregression__C': 10, 'tfidfvectorizer__ngram_range': (1, 2)}


In [74]:
pickle.dump(grid2, open('imdb2022022-2.pkl', 'wb'))

In [75]:
#Predizendo o modelo
y_pred = grid.predict(X_test)
score = grid.score(X_test, y_test)
print(score)

0.8432


In [76]:
pickle_in = open('imdb2022022-2.pkl', 'rb')
classifier = pickle.load(pickle_in)

In [77]:
teste=pd.read_csv('imdb_sample10.csv')
teste

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",pos
49455,O enredo teve algumas reviravoltas infelizes e...,pos
49456,Estou espantado com a forma como este filme e ...,pos
49457,A Christmas Together realmente veio antes do m...,pos


In [78]:
y_pred2=classifier.predict(teste)
y_pred2

array([1, 1])

In [38]:
teste.sentiment

0        neg
1        neg
2        neg
3        neg
4        neg
        ... 
49454    pos
49455    pos
49456    pos
49457    pos
49458    pos
Name: sentiment, Length: 49459, dtype: object

In [79]:
print(confusion_matrix(y_test, y_pred))

[[239  55]
 [ 43 288]]


In [80]:
print(pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0    1  All
Real                  
0        239   55  294
1         43  288  331
All      282  343  625


In [41]:
arquivo=pd.read_csv('imdb.csv')
arquivo

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",pos
49455,O enredo teve algumas reviravoltas infelizes e...,pos
49456,Estou espantado com a forma como este filme e ...,pos
49457,A Christmas Together realmente veio antes do m...,pos


In [42]:
arquivo.sample(100)

,text_pt,sentiment
47625,este filme é uma comédia romântica muito desco...,pos
21104,K-Pax é um filme muito intrigante. Prot Prot K...,pos
10339,"Este filme tem algumas boas atuações, como out...",neg
29760,"Como muitos filmes britânicos recentes, este l...",neg
21567,Esta é outra Universal divertida cheia susto f...,pos
...,...,...
45600,Apesar de ser contado a partir de uma perspect...,pos
5533,"Embora possa ser muito tedioso, às vezes, esse...",neg
31825,Esta série teria sido muito melhor se eles tiv...,neg
20312,"Eu concordo com outro revisor, este é um filme...",pos


In [43]:
arquivo_sample_1oo = arquivo.sample(100)

In [46]:
arquivo_sample_1oo.to_csv('arquivo_sample_100.csv', index=False)